In [41]:
# import os

# # Specify the directory
# directory = "../artifacts/models/columns.pkl"

# items = os.listdir(directory)

# print("Files and folders in the directory:")
# for item in items:
#     print(item)
!pip install --upgrade scikit-learn==1.5.2

In [17]:
import sklearn
print(sklearn.__version__)

1.5.2


In [18]:
import pickle
import sklearn

In [19]:
with open("../artifacts/models/columns.pkl", 'rb') as f:
    cols = pickle.load(f)

In [9]:
len(cols)

45

In [20]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_csv("../artifacts/data/processed/test.csv")

In [30]:
df.drop('price', axis=1).iloc[99].values

array(['Maurice Lacroix', 'Automatic', 39.0, 11.0, 'Round', 'Steel',
       'Sapphire Crystal', 'Steel', 'Folding', 'Men', 200.0, 2.0,
       'Switzerland', 38.0, 26.0, 0, 'No Bezel', 0, 28800.0, 'No coating',
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0], dtype=object)

In [26]:
np.random.randint(1, 3000)

994

In [12]:
test_input = pd.DataFrame([['Maurice Lacroix', 'Automatic', 39.0, 11.0, 'Round', 'Steel','Sapphire Crystal', 'Steel', 'Folding', 'Men', 
 200.0, 2.0, 'Switzerland', 38.0, 26.0, 0, 'No Bezel', 0, 28800.0, 'No coating',
1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]], columns=cols)

In [14]:
model.predict(test_input)

array([12.28554115])

In [21]:
import mlflow
import pytest
from mlflow.tracking import MlflowClient

# Set the tracking URI for your MLflow server
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
client = MlflowClient()

# Search for all runs of the model
runs = mlflow.search_runs(experiment_names=["pipeline-runs"])

# Sort runs by the start time (most recent first)
# runs = sorted(runs, key=lambda x: x.start_time, reverse=True)

In [28]:
df = pd.read_csv("../artifacts/data/processed/test.csv")
X = df.drop('price', axis=1)
y = np.log1p(df['price'])
# return X, Y

In [29]:
X

,Brand,Movement,Case Size,Case Thickness,Case Shape,Case Material,Glass Material,Strap Material,Clasp Type,Gender,...,feature_Moon Phase,feature_Date,feature_Small Seconds,feature_Triple Time-zone,feature_Month,feature_Perpetual Calendar,feature_Tourbillon,feature_Flying Tourbillon,feature_Day-Night Indicator,feature_Retrograde
0,TAG Heuer,Quartz,43.0,12.400000,Round,Steel,Sapphire Crystal,Steel,Folding,Men,...,0,1,1,0,0,0,0,0,0,0
1,Bovet,Manual Winding,42.0,9.050000,Round,Steel,Sapphire Crystal,Leather,Tang,Men,...,0,0,1,0,0,0,0,0,0,0
2,Luminox,Quartz,42.0,14.000000,Round,Carbon-based,Mineral Crystal,Rubber,Tang,Men,...,0,1,0,0,0,0,0,0,0,0
3,Corum,Automatic,47.0,18.500000,Round,Steel,Sapphire Crystal,Rubber,Tang,Men,...,0,0,0,0,0,0,0,0,0,0
4,Frederique Constant,Automatic,38.5,10.670000,Round,Steel,Sapphire Crystal,Leather,Tang,Men,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,Longines,Quartz,32.0,11.350000,Round,Ceramic,Sapphire Crystal,G-S Hybrid,Folding,Women,...,0,1,0,0,0,0,0,0,0,0
707,Luminox,Quartz,44.0,12.000000,Round,Steel,Sapphire Crystal,Rubber,Others,Men,...,0,1,0,0,0,0,0,0,0,0
708,Rado,Automatic,42.0,11.093343,Round,Ceramic,Sapphire Crystal,Ceramic,Others,Men,...,0,1,0,0,0,0,0,0,0,0
709,Jacob & Co.,Manual Winding,44.0,13.050000,Round,Steel,Sapphire Crystal,Rubber,Tang,Men,...,0,0,0,0,0,0,0,0,0,0


In [32]:
model_name = "watch_price_predictor"

client = MlflowClient()

candidate_versions = client.search_model_versions(f"name='{model_name}' and tags.deployment_status='staging'")
champion_versions = client.search_model_versions(f"name='{model_name}' and tags.deployment_status='champion'")

assert candidate_versions, "No staging model found for deployment."
candidate = candidate_versions[0]

if not champion_versions:
    assert candidate, "No champion model found for comparison."

    candidate_model_uri = f"models:/{model_name}/{candidate.version}"
    candidate_model = mlflow.pyfunc.load_model(candidate_model_uri)

    y_pred = candidate_model.predict(X)
    y_og = np.expm1(y) 
    y_pred_og = np.expm1(y_pred)

    mse = mean_squared_error(y_og, y_pred_og)
    candidate_rmse = np.sqrt(mse)
    candidate_mae = mean_absolute_error(y_og, y_pred_og)
    candidate_r2 = r2_score(y_og, y_pred_og)

#     assert candidate_rmse < 500000, f"RMSE of candidate model is too high: {candidate_rmse}"
#     assert candidate_mae < 200000, f"MAE of candidate model is too high: {candidate_mae}"
#     assert candidate_r2 > 85, f"R2 score of candidate model is too low: {candidate_r2}"
#     print(f"Candidate model meets the threshold with RMSE: {candidate_rmse}, MAE: {candidate_mae}, R2: {candidate_r2}")

else:
    champion = champion_versions[0]

    candidate_model_uri = f"models:/{model_name}/{candidate.version}"
    champion_model_uri = f"models:/{model_name}/{champion.version}"

    candidate_model = mlflow.pyfunc.load_model(candidate_model_uri)
    champion_model = mlflow.pyfunc.load_model(champion_model_uri)

    y_pred_candidate = candidate_model.predict(X)
    y_pred_champion = champion_model.predict(X)

    y_og = np.expm1(y)
    y_pred_candidate_og = np.expm1(y_pred_candidate)
    y_pred_champion_og = np.expm1(y_pred_champion)

    mse_candidate = mean_squared_error(y_og, y_pred_candidate_og)
    candidate_rmse = np.sqrt(mse_candidate)
    candidate_mae = mean_absolute_error(y_og, y_pred_candidate_og)
    candidate_r2 = r2_score(y_og, y_pred_candidate_og)

    mse_champion = mean_squared_error(y_og, y_pred_champion_og)
    champion_rmse = np.sqrt(mse_champion)
    champion_mae = mean_absolute_error(y_og, y_pred_champion_og)
    champion_r2 = r2_score(y_og, y_pred_champion_og)

#     assert candidate_rmse < champion_rmse, f"Candidate RMSE is higher than the champion RMSE: {candidate_rmse} > {champion_rmse}"
#     assert candidate_mae < champion_mae, f"Candidate MAE is higher than the champion MAE: {candidate_mae} > {champion_mae}"
#     assert candidate_r2 > champion_r2, f"Candidate R2 is lower than the champion R2: {candidate_r2} < {champion_r2}"

    print(f"Candidate model RMSE: {candidate_rmse}, MAE: {candidate_mae}, R2: {candidate_r2}")
    print(f"Champion model RMSE: {champion_rmse}, MAE: {champion_mae}, R2: {champion_r2}")


2024/12/06 14:53:25 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - logging (current: 0.5.1.2, required: logging==0.4.9.6)
 - numpy (current: 1.25.2, required: numpy==1.26.4)
 - pandas (current: 2.1.3, required: pandas==2.2.3)
 - psutil (current: 5.9.0, required: psutil==6.1.0)
 - scipy (current: 1.11.2, required: scipy==1.14.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


2024/12/06 14:53:25 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - logging (current: 0.5.1.2, required: logging==0.4.9.6)
 - numpy (current: 1.25.2, required: numpy==1.26.4)
 - pandas (current: 2.1.3, required: pandas==2.2.3)
 - psutil (current: 5.9.0, required: psutil==6.1.0)
 - scipy (current: 1.11.2, required: scipy==1.14.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


Candidate model RMSE: 292788.113047688, MAE: 101619.75721442133, R2: 0.9297959579385638
Champion model RMSE: 194562.67540178247, MAE: 59432.46428164642, R2: 0.9689991004892189


In [31]:
import mlflow.pyfunc
import pytest
from mlflow.tracking import MlflowClient
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score